# **Particle identification for the hadronic calorimeter imaging prototype**

### **CALICE AHCAL prototype**

The prototype has been installed in the test beam for data taking at the CERN SPS. During two periods in May and in June 2018, 
events with muon tracks, as well as electron and pion showers in the energy ranges 10 – 100 GeV and 10 – 200 GeV, respectively, 
have been recorded. 

In this seminar, data corresponding to simulation of [a highly granular SiPM-on-tile calorimeter prototype](https://arxiv.org/pdf/1808.09281.pdf) is used.

### **Setup**
38 active layers equipped with 24x24 scindilator plates ${30x30x2 mm^3}$ each equipped with individual SiPM readout. 

<img src="images/setup_1.jpg" alt="drawing" width="800"/>

Active layers are inserted in between ${15 mm}$ steel plates:

<img src="images/setup_2.jpg" alt="drawing" width="800"/>

### **Particle identification**

<img src="images/pid.jpg" alt="drawing" width="600"/>

If one wants to investigate detector response on particles with given properties, event selection is needed. The goal of the seminar is to construct such event selection model for 10GeV particles, which would be able to classify them according to their type using (almost) raw simulated hits in calorimeter layers.

We will use Boosted Decision Tree model for that purpose and specifically [LightGBM](https://lightgbm.readthedocs.io/en/latest/) implementation of it as an example. However, you might want to try other other popular boosting libraries: [XGBoost](https://xgboost.readthedocs.io/en/latest/) and [CatBoost](https://catboost.ai/docs/).

In [ ]:
import uproot
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D

### **Data structure**
In the dataset we are going to use each entry corresponds to a single event, while each subentry represents one single hit. Hit positions `ahc\_hitX/Y/Z` in ${mm}$  - Z-axis is along beam direction - and corresponding energy deposition `ahc\_hitEnergy` in units of Minimum Ionising Particle (MIP) $\approx 0.5 \, \text{MeV}$ are also stored.

In [ ]:
# Loading simulated data from the ROOT files to the Pandas dataframe
data = {}

variables = ['ahc_hitX','ahc_hitY','ahc_hitZ','ahc_hitEnergy','ahc_st']
data['Electron'] = uproot.open("data/Reco_Run60973_e-_10GeV_QGSP_BERT_HP_AllEvents.root")["bigtree"].pandas.df(variables) # ~10k events
data['Pion']  = uproot.open("data/Reco_Run60973_pi-_10GeV_QGSP_BERT_HP_AllEvents.root")["bigtree"].pandas.df(variables) # ~10k events
data['Muon']  = uproot.open("data/Reco_Run60973_mu-_10GeV_QGSP_BERT_HP_AllEvents.root")["bigtree"].pandas.df(variables) # ~10k events

data['Electron']['ID'] = 0 # Labels for the BDT model (chosen arbitrary)
data['Pion']['ID'] = 1
data['Muon']['ID'] = 2

particletypes = ['Electron', 'Pion', 'Muon']

#### **Exercise 1**

* Plot several events (at least 5) for different particle types, explain the difference
* How would you describe this visible difference quantatively? Feel free to modify the code below.

In [ ]:
# your answer here

In [ ]:
evt = 0
fig = plt.figure(figsize=[60,15])
for k in range(len(particletypes)):
    ax = fig.add_subplot(1, 3, k+1, projection='3d')
    ax.set_xlim3d(-300, 300)
    ax.set_ylim3d(-300, 300)
    ax.set_zlim3d(0,1000)
    ax.set_title(particletypes[k], fontsize=30)
    im = ax.scatter(data[particletypes[k]].loc[evt].ahc_hitX, data[particletypes[k]].loc[evt].ahc_hitY, data[particletypes[k]].loc[evt].ahc_hitZ, \
                  s=40, c=data[particletypes[k]].loc[evt].ahc_hitEnergy, depthshade=True,  cmap='viridis', vmin=0, vmax=10)
    fig.colorbar(im)
plt.show()

Or interactive mode:

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

evt = 1

fig = make_subplots(rows=1, cols=3,
                    specs=[[{'type': 'scene'}, {'type': 'scene'},  {'type': 'scene'}],
                          ],
                    subplot_titles=particletypes,
                    horizontal_spacing = 0.15
                   )

for i, particle_type in enumerate(particletypes, start=1):
    x = data[particle_type].loc[evt].ahc_hitX
    y = data[particle_type].loc[evt].ahc_hitY
    z = data[particle_type].loc[evt].ahc_hitZ
    e = data[particle_type].loc[evt].ahc_hitEnergy
    colorbar = dict(title = 'Hit energy deposit,\n [MIP]', len=0.75,) # x = ..., y = ...
    showscale = [False, False, True]
    
    fig.add_trace(
        go.Scatter3d(
        name=particle_type,
        x=x,
        y=y,
        z=z,
#         labels=dict(x="x, mm", y="y, mm", z="z, mm"),
        mode='markers',
        marker=dict(
            size=4,
            color=e,   
            cmin=0, cmax=20,
            colorscale='Viridis',  
            colorbar = colorbar,        
            opacity=1.,
            showscale=showscale[i-1]
        ),   
    ),
        row=1, col=i)

scenedict = dict(
            xaxis = dict(nticks=10, range=[-300,300], title = 'x, mm'),
            yaxis = dict(nticks=10, range=[-300,300], title = 'y, mm'),
            zaxis = dict(nticks=10, range=[0,1000], title = 'z, mm'),
            aspectratio=dict(x=0.6, y=0.6, z=1),
                    )
    
fig.update_layout(
        scene1 = scenedict,
        scene2 = scenedict,
        scene3 = scenedict,
)

fig.update_xaxes(title_text='x, mm')

fig.update_layout(
    title_text='Event display',
    height=800,
    width=1600,
    showlegend=False,
)


fig.show()

### **Some theory (in a very nutshell)**

Energy depositions of electromagnetic and hadronic showers are described by material-dependent parameters - so called [radiation- and nuclear interaction lengths](http://www.physics.rutgers.edu/~evahal/talks/tasi09/TASI_day3_school.pdf).  

In case of steel absorber, radiation lenth ${X_0}$ is ${O(1)}$ is less than nuclear interaction length ${\lambda_n}$ which, for example, leads to the fact that electromagnetic showers in the AHCAL start to shower much earlier than hadronic ones.

#### **Exercise 2**

* Can you draw any other conclusions from this?

In [ ]:
# your answer here

Shower start finding algorithm of CALICE software returns interger number of layer `ahc_st` where start of a shower is observed. Shower start variable was calculated for all events, in case of event that doesn't satisfy method criteria (shower was not found), shower start variable is set to 100. `ahc_st` is already read from the simulation ROOT file and written in Pandas DF and ready to be used in the event selection. Let's check how it looks for our events:

In [ ]:
for particle in particletypes:
    plt.hist(data[particle].groupby(level=0).ahc_st.mean(), range=[0,100], bins=100, log=True, alpha=0.5, label=particle)
fig1 = plt.gcf()
plt.title("Shower start")
plt.legend(loc='best')
plt.show()

### **Data preprocessing**

As we will see below, one observable is actually already quite enough to build a decent BDT model. Let's preprocess data a bit:

In [ ]:
# change event indices to avoid merging of events with same index
data['Pion'].index.set_levels(data['Pion'].index.levels[0]+10000, 
                              level=0,
                              inplace=True)
data['Muon'].index.set_levels(data['Muon'].index.levels[0]+20000, 
                              level=0,
                              inplace=True)

data_merged = pd.concat(data).droplevel(0)

In [ ]:
# creating Pandas dataframe on event-by-event basis
X = pd.DataFrame()

X['st'] = data_merged.groupby(level=0).ahc_st.mean()
X['ID'] = data_merged.groupby(level=0).ID.mean() 
features = ['st']
y = data_merged.groupby(level=0).ID.mean()

In [ ]:
# split data into train and test samples
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.5, random_state=42) #50% fraction may be useful to plot distributions for selected events

In [ ]:
# loading data into LightGMB format
lgb_train = lgb.Dataset(X_train[features], y_train)
lgb_eval = lgb.Dataset(X_test[features], y_test, reference=lgb_train)

### **Training model**

In [ ]:
# set model parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'softmax',
    'metric': {'multi_logloss'},
    'num_class': 3,
    'max_depth': 10,
    'metric_freq': 1,
    'num_leaves': 10,
    'max_depth' : 10,
    'min_child_samples' : 10,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [ ]:
# train the model

# Note: training can also be done with conventional fit() method, smth like this:
# classifier = lgb.LGBMClassifier(n_estimators=100)
# classifier.fit(X_train[features], y_train)

classifier = lgb.LGBMClassifier()
evals_result = {} 
classifier._Booster = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=[lgb_train, lgb_eval],
                evals_result = evals_result,
                early_stopping_rounds=5)
gbm = classifier._Booster

### **Model performance**

In [ ]:
# plot the loss function during training
ax = lgb.plot_metric(evals_result, metric='multi_logloss')
fig1 = plt.gcf()
plt.show()

In [ ]:
# make predictions with the model
y_pred = gbm.predict(X_test[features], num_iteration=gbm.best_iteration)

In [ ]:
# plot the most interesting ROC curves
for i in range(len(particletypes)):
    tpr, fpr, tres = roc_curve(y_test == i, y_pred[...,i])
    auc = roc_auc_score(y_test == i, y_pred[...,i])
    plt.plot(tpr, fpr, label=particletypes[i] + ' vs all \n AUC='+str(round(auc, 5)))
plt.grid()
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc='best')
fig1 = plt.gcf()
plt.show()

#### **Exercise 3**

Not bad for only one variable. Here are some questions: 
* Can you explain why is it so?
* What kind of events confuse such model and cause it to mispredict?

In [ ]:
# your answer here

### **Making predictions**

Finally, let's plot the classifier's output (aka score) in categories, which are defined by the maximum output value:

In [ ]:
df_pred = pd.DataFrame(y_pred, columns=['e_prob', 'pi_prob', 'mu_prob'])

df_pred['pred_type'] = np.argmax(y_pred, axis=1)
df_pred['pred_prob'] = np.max(y_pred, axis=1)

for i, particle_type in enumerate(particletypes):
    plt.hist(df_pred.query(f'pred_type == {i}').pred_prob, label=particle_type, histtype='step', bins=100, range=(0.5, 1.), density=True)
plt.grid()
plt.legend(loc='upper left')
plt.show()

In [ ]:
# interesting, isn't it?

### **Saving model**

Model can be saved, e.g in `.txt` format, for the later usage in your analysis group software framework with the corresponding API. 
Also note that some libraries [can save trees in if-else format](https://catboost.ai/docs/features/export-model-to-python.html), but not for the multiclass case.

In [ ]:
gbm.save_model("Multiclass_10-200GeV_MC_v2.txt")

### **Homework (main part)**

Good, now we've got the model baseline. There are several ideas how one can improve it: 

1. Create new features
2. Preprocess data in a clever way
3. Tune model hyperparameters 
4. Try other models (e.g. Random Forest, linear models, Neural Nets) 

So now we suggest you to do exactly that: try to beat our baseline! We are very curious to see which tricks and ideas helped you to achieve better performance:)